### **STAGE I — Data Preparation and Standardization**

Test Reader OK

In [1]:
my_label_csv_path = "/Users/alexeidelgado/Desktop/mpgcn-playground-scenes/data/processed/mpgcn_labels.csv"

In [2]:
from reader.playground_reader import Playground_Reader

reader = Playground_Reader(
    dataset_root_folder="/Users/alexeidelgado/Desktop/mpgcn-playground-scenes/data/npy",
    out_folder="/Users/alexeidelgado/Desktop/mpgcn-playground-scenes/workdir",
    label_csv_path= my_label_csv_path,
    num_frame=48,
    max_person=4,
    n_obj_max=None,
    max_joint=17,
    max_channel=3,
    split_strategy="auto"
)

In [3]:
reader.start()

Starting dataset build from CSV...

=== FILE DISCOVERY ===
Found 1146 pose files in /Users/alexeidelgado/Desktop/mpgcn-playground-scenes/data/npy
✅ All pose files have at least one matching object file.

Generating train split...


Scanning object files in /Users/alexeidelgado/Desktop/mpgcn-playground-scenes/data/npy: 100%|██████████| 1146/1146 [00:00<00:00, 16557.09it/s]


Max number of object nodes in /Users/alexeidelgado/Desktop/mpgcn-playground-scenes/data/npy: 7
Using n_obj_max = 7


100%|██████████| 1146/1146 [00:00<00:00, 5832.86it/s]


Unique object shapes found for train: {(48, 7, 3)}
Train set saved → /Users/alexeidelgado/Desktop/mpgcn-playground-scenes/workdir

=== FILE DISCOVERY ===
Found 1146 pose files in /Users/alexeidelgado/Desktop/mpgcn-playground-scenes/data/npy
✅ All pose files have at least one matching object file.

Generating eval split...


Scanning object files in /Users/alexeidelgado/Desktop/mpgcn-playground-scenes/data/npy: 100%|██████████| 1146/1146 [00:00<00:00, 23214.81it/s]


Max number of object nodes in /Users/alexeidelgado/Desktop/mpgcn-playground-scenes/data/npy: 7
Using n_obj_max = 7


100%|██████████| 1146/1146 [00:00<00:00, 20314.92it/s]

Unique object shapes found for eval: {(48, 7, 3)}
Eval set saved → /Users/alexeidelgado/Desktop/mpgcn-playground-scenes/workdir


In [8]:
import numpy as np
import pickle
import os

out_dir = "./../workdir"

# Load pose and object arrays
train_pose = np.load(os.path.join(out_dir, "train_data.npy"))
train_obj = np.load(os.path.join(out_dir, "train_object_data.npy"))
eval_pose = np.load(os.path.join(out_dir, "eval_data.npy"))
eval_obj = np.load(os.path.join(out_dir, "eval_object_data.npy"))

# Load labels
with open(os.path.join(out_dir, "train_label.pkl"), "rb") as f:
    train_labels = pickle.load(f)
with open(os.path.join(out_dir, "eval_label.pkl"), "rb") as f:
    eval_labels = pickle.load(f)

print("=== TRAIN SET ===")
print("Pose shape :", train_pose.shape)
print("Object shape :", train_obj.shape)
print("Labels :", len(train_labels))

print("\n=== EVAL SET ===")
print("Pose shape :", eval_pose.shape)
print("Object shape :", eval_obj.shape)
print("Labels :", len(eval_labels))

=== TRAIN SET ===
Pose shape : (909, 48, 4, 17, 3)
Object shape : (909, 48, 7, 3)
Labels : 909

=== EVAL SET ===
Pose shape : (237, 48, 4, 17, 3)
Object shape : (237, 48, 7, 3)
Labels : 237


In [9]:
print("Sample:", train_labels[0])

Sample: [6, 'trimmed_columpioscam1-2024-11-14_19_10_18_00-00-24_to_00-00-28_840']


In [10]:
idx = 5
print("Pose sample mean:", np.mean(train_pose[idx]))
print("Object sample mean:", np.mean(train_obj[idx]))
print("Label:", train_labels[idx])

Pose sample mean: 0.046615984
Object sample mean: 0.18753585
Label: [6, 'trimmed_columpioscam1-2024-11-16_21_51_42_00-00-11_to_00-00-15_217']


In [11]:
unique_pose_shapes = {x.shape for x in train_pose}
unique_obj_shapes = {x.shape for x in train_obj}
print(unique_pose_shapes, unique_obj_shapes)

{(48, 4, 17, 3)} {(48, 7, 3)}


In [12]:
import pandas as pd
label_ids = [l[0] for l in train_labels]
pd.Series(label_ids).value_counts().sort_index()

6    909
Name: count, dtype: int64

## **STAGE II — Graph Definition & Feature Generation**